<h1> HOMEWORK 2 </h1>
<h3> Background Information on data </h3>
<p>The dataset was dervived from an animal shelter in Austin, Texas. The dataset contains information regarding the animals that was brought to the shelter. The data includes final outcome of animal whether it was succesfully adopted/returned to owner or if animal died at the shelter. The outcome of 1 indicates death while 0 indicates that animal was adopted/returned</p>
The main objective in this homework is to build and evaluvate models for the animal shelter dataset. Find out how descriptive features are related to final outcome and to see if we can improve the model. The final predicition is <b>wheather an animal in the end have a positive or negative outcome.</b>

<h2>Part 1 Data Understanding and Preperation</h2>
    <p>Objective</p>
    <ul><li>Import all required libraries</li>
    <li> Import the cleaned csv file and review it
    <li>Explore relationship between feature pairs and transform them to meet requirements of training</li>
        <li>Shuffle data and split them into test and train data.</li>
    </ul>
        


<h3>1. Import libraries</h3>
<p> The most important library used is the <b>pandas</b>, this enables the csv to be converted to a pandas datatype and allow data manipulation. <b>Numpy</b> library adds array like functionalities to python for scientific calculations. The visulaization is carried out using <b>Matplotlib</b> and <b>Seaborn</b>.

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

This is done to ignore warnings generated by Jupyter Notebook and improve readablity

Scikit-learn is the package that will be used to train the various models. Its an open source Ml library which can be used for regression , classfication, random forest , spliting of data and cross-validation. It is designed to work well with numpy 

In [ ]:
from patsy import dmatrices
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.tree import export_graphviz

Graph viz will be used here to visualize trees

In [ ]:
import graphviz
from graphviz import Source

In [5]:
import warnings
warnings.simplefilter('ignore')

In [6]:
# This option enables auto complete in notebook
%config IPCompleter.greedy=True

In [7]:
# Load csv to pandas dataframe
df = pd.read_csv('AnimalWelfareRisk-19200494.csv')
# Enables viewing of all columns when using the head and tail command
pd.options.display.max_columns = None
pd.options.display.max_rows = None
# Show full column with out truncation
pd.set_option('display.max_colwidth', -1)

<p>The data that is going to be used in this homework was already prepared and cleaned in homework 1 </p>
<ul>
    <li>Additional information on the prepared data can be found in dataquality report provided along with the homework</li>
    <li>A pdf version of homework 1 notebook is also provided </li>
    <li>Summary of data quality report can be viewed in table below</li>
    </ul>
    
 <h3>Summary of data quality plan:</h3>

| ATTRIBUTE 	| DATA QUALITY ISSUE 	| HANDLING STRATEGY 	|
|---------------------	|:------------------------------------------------------------------------------------------------:	|----------------------------------------------------------------------------------------	|
| Animal ID 	| Data quality satisfactory. No useful information provide 	| Dropped 	|
| Name_Intake 	| Some values missing (NaN). Imputations  not possible.No useful information provided. 	| Dropped 	|
| DateTime_Intake 	| Data quality satisfactory. High cardinality. 	| Time data removed, Additional data extracted (AM/PM) and date 	|
| MonthYear_Intake 	| Duplicate of column 'DateTime_Intake' 	| Dropped 	|
| Found Location 	| Data satisfactory , High cardinality 	| Street wise data removed and only  city information retained. 	|
| Intake Type 	| Data quality satisfactory 	| No action required 	|
| Intake Condition 	| Data quality satisfactory 	| No action required 	|
| Animal Type_Intake 	| Data quality satisfactory 	| No action required 	|
| Sex upon Intake 	| Data quality satisfactory 	| No action required 	|
| Age upon Intake 	| Data quality satisfactory, Multiple units used 	| Convert all data to single unit of time 	|
| Breed_Intake 	| Data quality satisfactory, High cardinality 	| Data with less frequency binned to others category 	|
| Color_Intake 	| High cardinality, ambiguous information, Same data with different name. 	| Dropped 	|
| Name_Outcome 	| Duplicate of column 'Name_Intake'.Some data missing 	| Dropped 	|
| DateTime_Outcome 	| Data inconsistency , some dates lower than possible 	| Dates which show inconsistency imputed to same value of date in column DateTime_Intake 	|
| MonthYear_Outcome 	| Duplicate of column 'DateTime_Outcome' 	| Dropped 	|
| Date of Birth 	| Data quality satisfactory; However same info extractable  from other columns.Therefor redundant. 	| Dropped 	|
| Animal Type_Outcome 	| Duplicate of column 'Animal Type_Intake' 	| Dropped 	|
| Sex upon Outcome 	| Data quality satisfactory , same as column 'Sex upon Intake' 	| Dropped 	|
| Age upon Outcome 	| Data quality satisfactory 	| No action required 	|
| Breed_Outcome 	| Duplicate of colum "Breed Intake' 	| Dropped 	|
| Color_Outcome 	| High cardinality, ambiguous information , Same data with different name. 	| Dropped 	|
| binary_outcome 	| Wrong datatype of float for type binary outcome 	| Convert from Float to Categorical 	|

<p>Apart from above mentioned steps taken. Three new features where added by combining the attributes. This was done to better capture the problem domain</p>
They are:
<ul>
    <li>Seasons</li>
    <li>Time spend in shelter </li>
    <li>Domestic factor</li>
</ul>
The explanation for each new attribute is present in pdf of homework 1 

##  1.1 Review, prepare and split the dataset into two datasets: 70% training and 30% test
Since we have presented the data quality report. We will now import the cleaned csv file inspect it and see if there are still any defects present

In [9]:
#Reading the csv 
# parameters used in read_csv
# keep_default_na: Whether or not to include the default NaN values when parsing the data
# sep : Delimiter to use 
# skipinitialspace : Skip spaces after delimiter.
df = pd.read_csv("final.csv", keep_default_na=True, sep=',\s+', delimiter=',', skipinitialspace=True)

Lets look at data in brief

In [10]:
df.head(5)

,DateTime_Intake,Found Location,Intake Type,Intake Condition,Animal Type_Intake,Sex upon Intake,Age upon Intake,Breed_Intake,Sex upon Outcome,Age upon Outcome,binary_outcome,index,intake_am_pm,intake_time,percent,domesticIndex,season,time_spend_shelter
0,11/05/2016 12:12:00 PM,Austin (TX),Stray,Normal,Cat,Neutered Male,5,Domestic Shorthair Mix,Neutered Male,5,0.0,0,PM,2016-11-05,0.384615,0.900,Quarter 3,0.146658
1,09/28/2016 11:55:00 AM,Austin (TX),Public Assist,Normal,Dog,Intact Male,60,Pit Bull,Intact Male,60,0.0,1,AM,2016-09-28,14.285714,0.480,Quarter 3,0.084014
2,06/05/2015 05:18:00 PM,Travis (TX),Stray,Nursing,Cat,Intact Female,1,Domestic Shorthair Mix,Spayed Female,1,0.0,2,PM,2015-06-05,0.384615,0.432,Quarter 2,-2.167435
3,05/29/2015 11:45:00 PM,Austin (TX),Wildlife,Normal,Other,Unknown,1,Bat,Unknown,1,1.0,3,PM,2015-05-29,6.666667,0.000,Quarter 2,-1.766474
4,12/29/2015 12:58:00 PM,Austin (TX),Stray,Normal,Dog,Intact Male,1,Labrador Retriever Mix,Intact Male,2,0.0,4,PM,2015-12-29,2.000000,0.540,Quarter 3,-0.765618


In [13]:
df.head()

,Animal ID,Name_Intake,DateTime_Intake,MonthYear_Intake,Found Location,Intake Type,Intake Condition,Animal Type_Intake,Sex upon Intake,Age upon Intake,Breed_Intake,Color_Intake,Name_Outcome,DateTime_Outcome,MonthYear_Outcome,Date of Birth,Animal Type_Outcome,Sex upon Outcome,Age upon Outcome,Breed_Outcome,Color_Outcome,binary_outcome
0,A737831,NaN,11/05/2016 12:12:00 PM,11/05/2016 12:12:00 PM,3604 Alta Ct in Austin (TX),Stray,Normal,Cat,Neutered Male,5 months,Domestic Shorthair Mix,White/Orange Tabby,NaN,11/30/2016 02:23:00 PM,11/30/2016 02:23:00 PM,06/05/2016,Cat,Neutered Male,5 months,Domestic Shorthair Mix,White/Orange Tabby,0.0
1,A637559,Blue,09/28/2016 11:55:00 AM,09/28/2016 11:55:00 AM,Austin (TX),Public Assist,Normal,Dog,Intact Male,5 years,Pit Bull,Blue/White,Blue,10/08/2016 11:21:00 AM,10/08/2016 11:21:00 AM,09/19/2011,Dog,Intact Male,5 years,Pit Bull,Blue/White,0.0
2,A704521,*Goldie,06/05/2015 05:18:00 PM,06/05/2015 05:18:00 PM,5915 Old Lockhart Rd in Travis (TX),Stray,Nursing,Cat,Intact Female,1 year,Domestic Shorthair Mix,Orange Tabby,*Goldie,06/24/2015 04:41:00 PM,06/24/2015 04:41:00 PM,06/05/2014,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Orange Tabby,0.0
3,A703884,NaN,05/29/2015 11:45:00 PM,05/29/2015 11:45:00 PM,7102 Darcus Cv in Austin (TX),Wildlife,Normal,Other,Unknown,4 weeks,Bat,Black/Brown,NaN,05/31/2015 08:44:00 AM,05/31/2015 08:44:00 AM,04/29/2015,Other,Unknown,4 weeks,Bat,Black/Brown,1.0
4,A718401,NaN,12/29/2015 12:58:00 PM,12/29/2015 12:58:00 PM,1201 Rutland Dr in Austin (TX),Stray,Normal,Dog,Intact Male,1 month,Labrador Retriever Mix,Black/White,NaN,01/02/2016 12:09:00 PM,01/02/2016 12:09:00 PM,10/29/2015,Dog,Intact Male,2 months,Labrador Retriever Mix,Black/White,0.0


In [2]:
data = pd.read_csv(')

In [3]:
data.head()

,Animal ID,Name_Intake,DateTime_Intake,MonthYear_Intake,Found Location,Intake Type,Intake Condition,Animal Type_Intake,Sex upon Intake,Age upon Intake,...,Name_Outcome,DateTime_Outcome,MonthYear_Outcome,Date of Birth,Animal Type_Outcome,Sex upon Outcome,Age upon Outcome,Breed_Outcome,Color_Outcome,binary_outcome
0,A815848,NaN,03/27/2020 06:54:00 PM,03/27/2020 06:54:00 PM,2815 Manor Rd in Austin (TX),Wildlife,Normal,Other,Unknown,1 year,...,NaN,03/27/2020 07:00:00 PM,03/27/2020 07:00:00 PM,03/27/2019,Other,Unknown,NaN,Bat,Brown,1.0
1,A815835,NaN,03/27/2020 02:10:00 PM,03/27/2020 02:10:00 PM,12176 N Mopac Expy in Austin (TX),Wildlife,Normal,Other,Unknown,2 years,...,NaN,03/27/2020 03:12:00 PM,03/27/2020 03:12:00 PM,03/27/2018,Other,Unknown,NaN,Bat,Brown,1.0
2,A815834,NaN,03/27/2020 01:48:00 PM,03/27/2020 01:48:00 PM,715 E 8Th St in Austin (TX),Wildlife,Normal,Other,Unknown,2 years,...,NaN,03/27/2020 03:12:00 PM,03/27/2020 03:12:00 PM,03/27/2018,Other,Unknown,NaN,Bat,Brown,1.0
3,A815821,NaN,03/26/2020 11:45:00 PM,03/26/2020 11:45:00 PM,3621 Bratton Heights Dr in Austin (TX),Wildlife,Normal,Other,Unknown,2 years,...,NaN,03/27/2020 09:08:00 AM,03/27/2020 09:08:00 AM,03/26/2018,Other,Unknown,2 years,Bat,Brown/Brown,1.0
4,A815816,NaN,03/26/2020 11:21:00 PM,03/26/2020 11:21:00 PM,7801 Capital Of Texas in Austin (TX),Wildlife,Normal,Other,Unknown,2 years,...,NaN,03/27/2020 09:05:00 AM,03/27/2020 09:05:00 AM,03/26/2018,Other,Unknown,2 years,Bat,Brown/Brown,1.0


In [4]:
data.tail()

,Animal ID,Name_Intake,DateTime_Intake,MonthYear_Intake,Found Location,Intake Type,Intake Condition,Animal Type_Intake,Sex upon Intake,Age upon Intake,...,Name_Outcome,DateTime_Outcome,MonthYear_Outcome,Date of Birth,Animal Type_Outcome,Sex upon Outcome,Age upon Outcome,Breed_Outcome,Color_Outcome,binary_outcome
995,A813648,Ari,02/15/2020 11:04:00 AM,02/15/2020 11:04:00 AM,Austin (TX),Owner Surrender,Normal,Other,Intact Male,2 years,...,Ari,02/16/2020 03:22:00 PM,02/16/2020 03:22:00 PM,02/15/2018,Other,Intact Male,2 years,Hedgehog,Brown,0.0
996,A813646,NaN,02/15/2020 10:43:00 AM,02/15/2020 10:43:00 AM,5201 Ross Road in Travis (TX),Stray,Normal,Dog,Intact Female,3 years,...,NaN,02/20/2020 04:47:00 PM,02/20/2020 04:47:00 PM,02/15/2017,Dog,Spayed Female,3 years,Chihuahua Longhair,White,0.0
997,A813644,Puddin,02/15/2020 10:32:00 AM,02/15/2020 10:32:00 AM,10000 Slaughter Creek Drive in Austin (TX),Stray,Normal,Dog,Spayed Female,7 years,...,Puddin,02/15/2020 02:23:00 PM,02/15/2020 02:23:00 PM,02/15/2013,Dog,Spayed Female,7 years,Labrador Retriever,Yellow,0.0
998,A813641,NaN,02/14/2020 08:47:00 PM,02/14/2020 08:47:00 PM,2121 Craig Patrick Dr in Austin (TX),Wildlife,Normal,Other,Unknown,2 years,...,NaN,02/15/2020 09:53:00 AM,02/15/2020 09:53:00 AM,02/14/2018,Other,Unknown,2 years,Bat,Brown/Brown,1.0
999,A813640,NaN,02/14/2020 08:17:00 PM,02/14/2020 08:17:00 PM,Harrisglenn in Austin (TX),Stray,Normal,Dog,Neutered Male,1 year,...,NaN,02/19/2020 05:24:00 PM,02/19/2020 05:24:00 PM,02/14/2019,Dog,Neutered Male,1 year,Chihuahua Shorthair,Black/White,0.0
